In [1]:
import os
os.chdir('A:\Documents and Data\R\Datasets-R\digot_recognizer')

C:\Users\Nuages\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
import pandas as pd
training_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('dig_rec_test.csv')

In [5]:
len(training_data), len(test_data), 64*525

(33602, 28000, 33600)

In [6]:
training_data[0:10].loc[training_data[0:10]['label'] == 0]

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
x_train = training_data.drop(['label'],1)
y_train = training_data['label']

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [18]:
N, D_in, H, D_out = 64, 784, 60, 10

In [10]:
x = torch.tensor(x_train.values, dtype=torch.float32)
y = torch.tensor(y_train.values)

In [19]:
x_test = torch.tensor(test_data.values, dtype=torch.float32)

In [11]:
class Network(nn.Module):
    
    def __init__(self, D_in, H, D_out):
        super(Network, self).__init__()
        self.linear1 = nn.Linear(D_in, H)
        self.linear2 = nn.Linear(H, D_out)
        
    def forward(self, inputs):
        out = torch.sigmoid(self.linear1(inputs))
        out = self.linear2(out)
        log_probs = torch.softmax(out, dim=1)
        return out

In [12]:
losses = []
loss_function = nn.CrossEntropyLoss()
model = Network(D_in, H, D_out)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [17]:
for i in range(50):
    for epoch in range(525):
        log_probs = model(x[epoch*N:(epoch+1)*N])
        loss = loss_function(log_probs, y[epoch*N:(epoch+1)*N])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss)
    print(losses[-1])

tensor(0.0620, grad_fn=<NllLossBackward>)
tensor(0.0616, grad_fn=<NllLossBackward>)
tensor(0.0611, grad_fn=<NllLossBackward>)
tensor(0.0607, grad_fn=<NllLossBackward>)
tensor(0.0605, grad_fn=<NllLossBackward>)
tensor(0.0602, grad_fn=<NllLossBackward>)
tensor(0.0600, grad_fn=<NllLossBackward>)
tensor(0.0596, grad_fn=<NllLossBackward>)
tensor(0.0597, grad_fn=<NllLossBackward>)
tensor(0.0590, grad_fn=<NllLossBackward>)
tensor(0.0585, grad_fn=<NllLossBackward>)
tensor(0.0582, grad_fn=<NllLossBackward>)
tensor(0.0580, grad_fn=<NllLossBackward>)
tensor(0.0580, grad_fn=<NllLossBackward>)
tensor(0.0576, grad_fn=<NllLossBackward>)
tensor(0.0574, grad_fn=<NllLossBackward>)
tensor(0.0572, grad_fn=<NllLossBackward>)
tensor(0.0570, grad_fn=<NllLossBackward>)
tensor(0.0569, grad_fn=<NllLossBackward>)
tensor(0.0566, grad_fn=<NllLossBackward>)
tensor(0.0562, grad_fn=<NllLossBackward>)
tensor(0.0560, grad_fn=<NllLossBackward>)
tensor(0.0556, grad_fn=<NllLossBackward>)
tensor(0.0554, grad_fn=<NllLossBac

In [20]:
y_pred = model(x_test)

In [27]:
predictions = []
for i in y_pred:
    predictions.append(torch.argmax(i).item())

In [58]:
pred = pd.DataFrame({'ImageId' : [i+1 for i in range(28000)], 'Label' : predictions})

In [59]:
pred[0:10]

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,2
5,6,7
6,7,0
7,8,3
8,9,0
9,10,3


In [60]:
sample = pd.read_csv('sample_submission.csv')

In [62]:
sample['Label'] = pred['Label']

In [63]:
pred.to_csv('pred.csv')